### Categories of Joins
The pd.merge() function implements a number of types of joins: one-to-one, many-to-one, many-to-many joins

### One-to-one join

In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.DataFrame({"employee": ["Bob", "Jake", "Lisa", "Sue", "Bob"], 
                    "group": ["Accounting", "Engineering", "Engineering", "HR", "Sales"]})

df2 = pd.DataFrame({"employee": ["Lisa", "Bob", "Jake", "Sue"],
                   "hire_date": [2004, 2008, 2012, 2014]})

df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
4,Bob,Sales


In [3]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


#### To combine this information into a single DataFrame, we can use the pd.merge() function 

In [4]:
df3 = pd.merge(df1, df2)

df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Bob,Sales,2008
2,Jake,Engineering,2012
3,Lisa,Engineering,2004
4,Sue,HR,2014


#### The pd,merge() function recognizes that each `DataFrame` has an "employee" column and automatically joins using the column as a key

### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries

In [5]:
df4 = pd.DataFrame({"group": ["Accounting", "Engineering", "HR"],
                   "supervisor": ["Carly", "Guido", "Steve"]})

pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


The resulting `DataFrame` has an additional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins

Many-to-many joins are bit confusing conceptually, but are nevertheless well defined. If the key column in both the left and right array contains duplicate,then the result is a many-to-many merge.

In [6]:
df5 = pd.DataFrame({"group": ["Accounting", "Accounting", "Engineering", "Engineering", "HR", "HR"],
                   "skills": ["math", "spreedsheets", "coding", "linux", "spreedsheets", "organization"]})

df6 = pd.merge(df1, df5)
df6

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreedsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreedsheets
7,Sue,HR,organization


### Specify the Merge Key

#### The `on` keyword

Most simply, you can explicitly specify the name of the key column using the `on` keyword which takes a column name or a list of column names.

In [7]:
df7 = pd.merge(df1, df2, on="employee") # This options works only if the left and right 'DataFrames' have the specified column name

df7

,employee,group,hire_date
0,Bob,Accounting,2008
1,Bob,Sales,2008
2,Jake,Engineering,2012
3,Lisa,Engineering,2004
4,Sue,HR,2014


### The left-on and right-on keywords

At times you may wish to merge two documents with disfferent column names. For example, we may have a dataset in which the employee name is labelled 'name' rather than 'employee', in this case we use the left-on and right-on keyword to specify the two column names 

In [8]:
df3 = pd.DataFrame({"name": ["Bob", "Jake", "Lisa", "Sue"],
                   "salary": [70000, 80000, 120000, 90000]})

df8 = pd.merge(df1, df3, left_on="employee", right_on="name")

df8

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Bob,Sales,Bob,70000
2,Jake,Engineering,Jake,80000
3,Lisa,Engineering,Lisa,120000
4,Sue,HR,Sue,90000


The result has a redundant column that we can drop if desired-for example, by using the drop() method `DataFrame`

In [9]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop("name", axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Bob,Sales,70000
2,Jake,Engineering,80000
3,Lisa,Engineering,120000
4,Sue,HR,90000


For convenience, we'll define this function which creates a `DataFrame` of a particular form that will be useful below:

## Combining Datasets: Concat and Append

### Simple Concatenation with `pd.concat`

`pd.concat` can be used for a simple concatenation of `Series` or `DataFrame`objects.

In [15]:
ser1 = pd.Series(["A", "B", "C"], index=[1, 2, 3])
ser2 = pd.Series(["D", "E", "F"], index=[4, 5, 6])

pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

It also works to concatenate hgher dimensional objects such as `DataFrame`s

In [16]:
df1 = pd.DataFrame({"AB": [1, 2]})
df2 = pd.DataFrame({"AB": [3, 4]})

pd.concat([df1, df2])

,AB
0,1
1,2
0,3
1,4


#### By default the concatenation takes place row-wise in `DataFrame` (i.e... `axis=0`). Like `np.concatenate`, `pd.concat` allows specification of an axis along which concatenation will take place. Consider the following example

In [17]:
df3 = pd.DataFrame({"AB": [0, 1]})
df4 = pd.DataFrame({"CD": [1, 0]})

pd.concat([df3, df4], axis=1)

,AB,CD
0,0,1
1,1,0


##### We could have equivalently specified `axis=1`; here we've used the more intuitive `axis='col'`

In [18]:
df3 = pd.DataFrame({"AB": [0, 1]})
df4 = pd.DataFrame({"CD": [1, 0]})

pd.concat([df3, df4], axis=0)

,AB,CD
0,0.0,NaN
1,1.0,NaN
0,NaN,1.0
1,NaN,0.0


##### We could have equivalently specified `axis=0`; here we've used the more intuitive `axis='row'`

### The `append()` method

In [19]:
df1

,AB
0,1
1,2


In [20]:
df2

,AB
0,3
1,4


In [21]:
df1.append(df2) # Note: use pandas.concat in future as frame.append will be removed from pandas

C:\Users\Charity\AppData\Local\Temp\ipykernel_12556\2477559626.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2) # Note: use pandas.concat in future as frame.append will be removed from pandas


,AB
0,1
1,2
0,3
1,4


## GroupBy

An essential piece of analysis of large data is efficient summarization; computing aggregations like `sum()`, `mean()`, `median()`, `min()` and `max()` in which a single number gives insight into the nature of a potentially large dataset 

In [22]:
df = pd.DataFrame({"key": ["A", "B", "C", "A", "B", "C"],
                  "data": range(6)}, columns=["key", "data"])

df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [23]:
df.index

RangeIndex(start=0, stop=6, step=1)

In [24]:
df.groupby("key")

#### Notice that what is returned is not a set of `Dataframe`s but a `DataFrameGroupby` object. This object is where the magic is: You can think of it as a special vie of the `DataFrame` which is poised to dig into the groups but does no actual computation until the aggregation is applied. This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is transpsrent to the user.

#### To produce a result, we can apply an aggregate to this `DataFrameGroupBy` object, which will perform the appropriate apply/combine steps to produce the desired result.

In [25]:
df.groupby("key")["data"].sum()

key
A    3
B    5
C    7
Name: data, dtype: int64

In [26]:
df.groupby("key").sum()

,data
key,
A,3
B,5
C,7
